<a href="https://colab.research.google.com/github/zia207/GCMS/blob/main/mc_ms_raw_data_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyMassSpec - Python Toolkit for Mass Spectrometry

Zia Ahmed, RENEW, University at Buffalo



[PyMassSpec](https://pymassspec.readthedocs.io/en/master/index.html#) is a Python package for processing gas chromatography-mass spectrometry data. PyMassSpec provides a framework and a set of components for rapid development and testing of methods for processing of chromatography–mass spectrometry data. PyMassSpec can be used interactively through the Python shell, in a Jupyter Notebook, or the functions can be collected into scripts when it is preferable to perform data processing in the batch mode.

In [4]:
! pip install PyMassSpec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.3 MB/s eta 0:00:00


## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. GC-MS Raw Data Wragling

**Table of Contents**

1.1. Introduction

1.2. Reading JCAMP GC-MS data

1.2.1 A GCMS_data Object

1.2.2. A Scan Object

1.2.3. Exporting data and obtaining information about a data set

1.3. Comparing two GC-MS data sets

### 1.1. Introduction

PyMassSpec can read gas chromatography-mass spectrometry (GC-MS) data stored in Analytical Data Interchange for Mass Spectrometry (ANDI-MS), 1 and Joint Committee on Atomic and Molecular Physical Data (JCAMP-DX) 2 formats. The information contained in the data files can vary significantly depending on the instrument, vendor’s software, or conversion utility. PyMassSpec makes the following assumptions about the information contained in the data file:

The data contain the m/z and intensity value pairs across a scan.

Each scan has a retention time.

Internally, PyMassSpec stores the raw data from ANDI files or JCAMP files as a GCMS_data object.

### 1. 2. Reading JCAMP GC-MS data

The PyMS package `pyms.GCMS.IO.JCAMP` provides capabilities to read the raw GC-MS data stored in the JCAMP-DX format.

First, setup the paths to the datafile and the output directory, then import JCAMP_reader.

In [43]:
import pathlib
data_directory =pathlib.Path("/content/drive/MyDrive/PyMassSpec_tutorial/data/")
# Change this if the data files are stored in a different location
output_directory =pathlib.Path("/content/drive/MyDrive/PyMassSpec_tutorial/output/")

In [25]:
from pyms.GCMS.IO.JCAMP import JCAMP_reader

In [26]:
jcamp_file = data_directory / "gc01_0812_066.jdx"
data = JCAMP_reader(jcamp_file)
data

 -> Reading JCAMP file '/content/drive/MyDrive/PyMassSpec_tutorial/data/gc01_0812_066.jdx'


<GCMS_data(305.582 - 4007.722 seconds, time step 0.3753183292781833, 9865 scans)>

### 1.2.1. A GCMS data Object
The object data (from the two previous examples) stores the raw data as a pyms.GCMS.Class.GCMS_data object. Within the GCMS_data object, raw data are stored as a list of pyms.Spectrum.Scan objects and a list of retention times. There are several methods available to access data and attributes of the GCMS_data and Scan objects.

The GCMS_data object’s methods relate to the raw data. The main properties relate to the `masses`, `retention times` and `scans`. For example, the minimum and maximum mass from all of the raw data can be returned by the following:

In [27]:
data.min_mass

50.0

In [28]:
data.max_mass

599.9

A list of the first 10 retention times can be returned with:g

In [29]:
data.time_list[:10]

[305.582,
 305.958,
 306.333,
 306.708,
 307.084,
 307.459,
 307.834,
 308.21,
 308.585,
 308.96]

The index of a specific retention time (in seconds) can be returned with:

In [30]:
data.get_index_at_time(400.0)

252

Note that this returns the index of the retention time in the data closest to the given retention time of 400.0 seconds.

The `GCMS_data.tic attribute` returns a total ion chromatogram (TIC) of the data as an `IonChromatogram object`:

In [31]:
data.tic

***Models an ion chromatogram***.

An ion chromatogram is a set of intensities as a function of retention time. This can can be either `m/z` channel intensities (for example, ion chromatograms at `m/z = 65`), or cumulative intensities over all measured `m/z`. In the latter case the ion chromatogram is total ion chromatogram (TIC).

The nature of an IonChromatogram object can be revealed by inspecting the value of the attribute ‘mass’. This is set to the `m/z` value of the ion chromatogram, or to None for TIC.

### 1.2.2. A Scan Object

A `pyms.Spectrum.Scan` object contains a list of masses and a corresponding list of intensity values from a single mass-spectrum scan in the raw data. Typically only non-zero (or non-threshold) intensities and corresponding masses are stored in the raw data.

A list of the first 10 `pyms.Spectrum.Scan` objects can be returned with:

In [34]:
scans = data.scan_list
scans[:10]

A list of the first 10 masses in a scan (e.g. the 1st scan) is returned with:

In [35]:
scans[0].mass_list[:10]

[50.1, 51.1, 53.1, 54.2, 55.1, 56.2, 57.2, 58.2, 59.1, 60.1]

A list of the first 10 corresponding intensities in a scan is returned with:

In [33]:
scans[0].intensity_list[:10]

[22128.0,
 10221.0,
 31400.0,
 27352.0,
 65688.0,
 55416.0,
 75192.0,
 112688.0,
 152256.0,
 21896.0]

The minimum and maximum mass in an individual scan (e.g. the 1st scan) are returned with:

In [36]:
scans[0].min_mass

50.1

In [37]:
scans[0].max_mass

599.4

### 1.2.3. Exporting data and obtaining information about a data set

Often it is of interest to find out some basic information about the data set, e.g. the number of scans, the retention time range, and m/z range and so on. The `GCMS_data` class provides a method `info()` that can be used for this purpose.

In [38]:
data.info()

 Data retention time range: 5.093 min -- 66.795 min
 Time step: 0.375 s (std=0.000 s)
 Number of scans: 9865
 Minimum m/z measured: 50.000
 Maximum m/z measured: 599.900
 Mean number of m/z values per scan: 56
 Median number of m/z values per scan: 40


The entire raw data of a GCMS_data object can be exported to a file with the method `write()`:

In [44]:
data.write(output_directory/"data")

 -> Writing intensities to '/content/drive/MyDrive/PyMassSpec_tutorial/output/data.I.csv'
 -> Writing m/z values to '/content/drive/MyDrive/PyMassSpec_tutorial/output/data.mz.csv'


This method takes the filename (“output/data”, in this example) and writes two CSV files. One has extension “.I.csv” and contains the intensities (“output/data.I.csv” in this example), and the other has the extension “.mz” and contains the corresponding table of m/z value (“output/data.mz.csv” in this example). In general, these are not two-dimensional matrices, because different scans may have different number of m/z values recorded.

## 1.3. Comparing two GC-MS data sets

Occasionally it is useful to compare two data sets. For example, one may want to check the consistency between the data set exported in netCDF format from the manufacturer’s software, and the JCAMP format exported from a third party software.

* **ANDI-MS** was developed by the Analytical Instrument Association

* **JCAMP-DX** is maintained by the International Union of Pure and Applied Chemistry

First, import `JCAMP_reader` and `ANDI_reader`.

In [45]:
from pyms.GCMS.IO.JCAMP import JCAMP_reader
from pyms.GCMS.IO.ANDI import ANDI_reader

Then the raw data is read as before.

In [46]:
andi_file = data_directory / "gc01_0812_066.cdf"
data1 = ANDI_reader(andi_file)
data1

 -> Reading netCDF file '/content/drive/MyDrive/PyMassSpec_tutorial/data/gc01_0812_066.cdf'


<GCMS_data(305.582 - 4007.721 seconds, time step 0.37531822789943226, 9865 scans)>

In [47]:
jcamp_file = data_directory / "gc01_0812_066.jdx"
data2 = JCAMP_reader(jcamp_file)
data2

 -> Reading JCAMP file '/content/drive/MyDrive/PyMassSpec_tutorial/data/gc01_0812_066.jdx'


<GCMS_data(305.582 - 4007.722 seconds, time step 0.3753183292781833, 9865 scans)>

To compare the two data sets, use the function `diff()`

In [48]:
from pyms.GCMS.Function import diff
diff(data1, data2)

 Data sets have the same number of time points.
   Time RMSD: 3.54e-04
 Checking for consistency in scan lengths ...OK
 Calculating maximum RMSD for m/z values and intensities ...
   Max m/z RMSD: 1.03e-05
   Max intensity RMSD: 0.00e+00


If the data cannot be compared, for example because of different number of scans, or inconsistent number of m/z values in between two scans, `diff()` will report the difference. For example:

In [49]:
data2.trim(begin=1000, end=2000)

Trimming data to between 1000 and 2001 scans


In [50]:
diff(data1, data2)

 The number of retention time points differ.
	First data set: 9865 time points
	Second data set: 1002 time points
 Data sets are different.
